# Views are different perspectives of our data
That would be a way to get a better understanding of the data.  
If you have an idea for a view, share it or make an issue.

In [1]:
import altair as alt
import numpy as np
import pandas as pd

from mlrepricer import match, setup, helper, schemas
from mlrepricer.minmax import dump_csv
from mlrepricer.database import SQLite
alt.data_transformers.enable('default', max_rows=1000000)

DataTransformerRegistry.enable('default')

In [58]:
t = schemas.pricemonitor(SQLite)()  # tableobject
df = pd.read_sql_query(f'SELECT * FROM {t.table}', t.conn, parse_dates=[t.eventdate], index_col='ID')

### Filter1 aka features we probably should not track in the first place.
for me it drops like 4% of the data

In [61]:
filter1 = df[(df.instock==1)&(df.isfeaturedmerchant==1)]
filter1 = filter1.drop(['instock', 'isfeaturedmerchant'], axis=1)

We look at each attribute separetely now.  
Basically it's ok to drop both features.  
The instock attribute has some winners, but it's to unlikely.  
It's like 1/300 of the dataset got that feature.

In [52]:
drop2 = df[df.isfeaturedmerchant==0]  # it has a zero chance to be a winner
print(f'We can drop {len(drop2)} rows and loose {len(drop2[drop2.isbuyboxwinner==1])} winners, we expected this to be Zero')

We can drop 10365 rows and loose 0 winners, we expected this to be Zero


In [53]:

drop = df[df.instock==0]
normalized_lostwinner = len(filter1[filter1.isbuyboxwinner==1])/len(filter1)/(len(drop[drop.isbuyboxwinner==1])/len(drop))
print(f'Its {normalized_lostwinner:.2f} times less likely to find a buyboxwinner in the data set we drop here, we have {len(drop)} datarows.')
if normalized_lostwinner < 5:
    raise ValueError

Its 6.19 times less likely to find a buyboxwinner in the data set we drop here, we have 1001 datarows.


## What is the name for the type of feed data we get?
It's like time series data, but tracking only changes.
I think without reducing the complexity we can imagine it in 5 dimensions.
 - asin
 - time_changed  
   competitor = groupby(['sellerid', 'isprime']).min()  # min because maybe some have duplicates offers  
     those are two seperate dimensions:
     - sellerid
     - isprime
 - features
 
does this make sense?

Knowing this we can make a pivot table. Let's refer to it as groundtruth.

In [62]:
filter1.groupby(['asin', 'sellerid', 'isprime', 'time_changed']).min().head(10)

feedback  \
asin       sellerid       isprime time_changed                        
B0126QIK2K A2EHWGAW6J9W8Q 0       2018-05-22 07:33:12.314      1606   
                                  2018-05-22 07:42:09.934      1606   
                                  2018-05-22 09:38:58.802      1606   
           AFEE1JGTCYXOG  0       2018-05-22 07:33:12.314      2072   
                                  2018-05-22 07:42:09.934      2072   
                                  2018-05-22 09:38:58.802      2072   
B015NJUI5O A10GXGUDASI5WW 1       2018-05-10 02:03:16.011       503   
                                  2018-05-10 06:03:58.780       503   
                                  2018-05-10 13:23:17.559       503   
                                  2018-05-10 20:30:04.405       504   

                                                           feedbackpercent  \
asin       sellerid       isprime time_changed                               
B0126QIK2K A2EHWGAW6J9W8Q 0       2018-05-22 07:33:12.314               98   
                                  2018-05-22 07:42:09.934               98   
                                  2018-05-22 09:38:58.802               98   
           AFEE1JGTCYXOG  0       2018-05-22 07:33:12.314              100   
                                  2018-05-22 07:42:09.934              100   
                                  2018-05-22 09:38:58.802              100   
B015NJUI5O A10GXGUDASI5WW 1       2018-05-10 02:03:16.011               99   
                                  2018-05-10 06:03:58.780               99   
                                  2018-05-10 13:23:17.559               99   
                                  2018-05-10 20:30:04.405               99   

                                                           isbuyboxwinner  \
asin       sellerid       isprime time_changed                              
B0126QIK2K A2EHWGAW6J9W8Q 0       2018-05-22 07:33:12.314               0   
                                  2018-05-22 07:42:09.934               1   
                                  2018-05-22 09:38:58.802               1   
           AFEE1JGTCYXOG  0       2018-05-22 07:33:12.314               1   
                                  2018-05-22 07:42:09.934               0   
                                  2018-05-22 09:38:58.802               0   
B015NJUI5O A10GXGUDASI5WW 1       2018-05-10 02:03:16.011               0   
                                  2018-05-10 06:03:58.780               1   
                                  2018-05-10 13:23:17.559               1   
                                  2018-05-10 20:30:04.405               0   

                                                           price  \
asin       sellerid       isprime time_changed                     
B0126QIK2K A2EHWGAW6J9W8Q 0       2018-05-22 07:33:12.314  12.00   
                                  2018-05-22 07:42:09.934  12.00   
                                  2018-05-22 09:38:58.802  12.00   
           AFEE1JGTCYXOG  0       2018-05-22 07:33:12.314  11.85   
                                  2018-05-22 07:42:09.934  21.57   
                                  2018-05-22 09:38:58.802  32.29   
B015NJUI5O A10GXGUDASI5WW 1       2018-05-10 02:03:16.011  10.48   
                                  2018-05-10 06:03:58.780  10.48   
                                  2018-05-10 13:23:17.559  10.48   
                                  2018-05-10 20:30:04.405  12.99   

                                                           shipping_maxhours  \
asin       sellerid       isprime time_changed                                 
B0126QIK2K A2EHWGAW6J9W8Q 0       2018-05-22 07:33:12.314                 48   
                                  2018-05-22 07:42:09.934                 48   
                                  2018-05-22 09:38:58.802                 48   
           AFEE1JGTCYXOG  0       2018-05-22 07:33:12.314                 24   
                                  2018-05-22 07

What can we improve from here?
 - only track stable states, like those hold for more than 5 minutes
 - linear transform isprime to remove a dimension  
More ideas are welcome on github.

In [ ]:
# 